# Task 2: Data Acquisition & Initial Visualization

## This is the .ipynb for Task 2:

You may use the provided USGS streamflow datasets or a time-series dataset of your own choosing, provided it has at least daily resolution.

Load Data: Read the raw data into a Pandas DataFrame from your  data/ folder.

Process the data: Subset the data so that all of the data is overlapping and is 6 years in length.
Exploratory Data Analysis (EDA): Create an initial time-series plot of the data, ensure the plot has a title, axes labels, legend. etc. Create a figure with four subplots to visualize the data.

Compare and Contrast: How do the different streams compare (timing, magnitude, other)?




In [2]:
# Import Packages

import pandas as pd
import matplotlib.pyplot as plt
import os

#### Load Data

In [3]:
# Example for one file
df = pd.read_csv("../data/raw/09289500_1980_2020.csv")
print(df.head())        # Shows first 5 rows
print(df.columns)       # Shows all column names


     Datetime  USGS_flow    variable  USGS_ID measurement_unit     qualifiers  \
0  1984-10-01      84.15  streamflow  9289500            ft3/s          ['A']   
1  1984-10-02        NaN  streamflow  9289500            ft3/s          ['A']   
2  1984-10-03        NaN  streamflow  9289500            ft3/s          ['A']   
3  1984-10-04        NaN  streamflow  9289500            ft3/s          ['A']   
4  1984-10-05        NaN  streamflow  9289500            ft3/s  ['A', '[91]']   

   series  
0       0  
1       0  
2       0  
3       0  
4       0  
Index(['Datetime', 'USGS_flow', 'variable', 'USGS_ID', 'measurement_unit',
       'qualifiers', 'series'],
      dtype='object')


In [4]:
# Define file paths and labels
files = {
    "Lake Fork River": "../data/raw/09289500_1980_2020.csv",
    "Green River": "../data/raw/09261000_1980_2020.csv",
    "Colorado River Potash": "../data/raw/09185600_1980_2020.csv",
    "Colorado River Cisco": "../data/raw/09180500_1980_2020.csv"
}

# Read CSVs into a dictionary of DataFrames
data = {label: pd.read_csv(path, parse_dates=['Datetime'], index_col='Datetime') 
        for label, path in files.items()}

# Quick check
for label, df in data.items():
    print(label, df.shape)
    print(df.head(3))
    print(" ")


Lake Fork River (13110, 6)
            USGS_flow    variable  USGS_ID measurement_unit qualifiers  series
Datetime                                                                      
1984-10-01      84.15  streamflow  9289500            ft3/s      ['A']       0
1984-10-02        NaN  streamflow  9289500            ft3/s      ['A']       0
1984-10-03        NaN  streamflow  9289500            ft3/s      ['A']       0
 
Green River (14610, 6)
            USGS_flow    variable  USGS_ID measurement_unit     qualifiers  \
Datetime                                                                     
1980-09-30  1320.0000  streamflow  9261000            ft3/s  ['A', '[91]']   
1980-10-01  2173.5417  streamflow  9261000            ft3/s  ['A', '[91]']   
1980-10-02  1421.6666  streamflow  9261000            ft3/s  ['A', '[91]']   

            series  
Datetime            
1980-09-30       0  
1980-10-01       0  
1980-10-02       0  
 
Colorado River Potash (2162, 6)
            USGS_flow  

#### Find most recent contiuous 6 year period

In [5]:
# Loop through datasets to inspect date ranges and gaps
for label, df in data.items():
    print(f"=== {label} ===")
    
    first_date = df.index.min()
    last_date  = df.index.max()
    print("First date:", first_date.date())
    print("Last date: ", last_date.date())
    
    # Compute gaps in days between consecutive rows
    gaps = df.index.to_series().diff().dt.days
    large_gaps = gaps[gaps > 7]
    
    if not large_gaps.empty:
        print("Gaps > 7 days found on these dates:")
        for date, gap in large_gaps.items():
            print(f"  {date.date()} (gap of {int(gap)} days)")
    else:
        print("No gaps > 7 days")
    
    print(" ")


=== Lake Fork River ===
First date: 1984-10-01
Last date:  2020-09-29
Gaps > 7 days found on these dates:
  1990-10-20 (gap of 39 days)
 
=== Green River ===
First date: 1980-09-30
Last date:  2020-09-29
No gaps > 7 days
 
=== Colorado River Potash ===
First date: 2014-10-30
Last date:  2020-09-29
No gaps > 7 days
 
=== Colorado River Cisco ===
First date: 1986-10-01
Last date:  2020-09-29
No gaps > 7 days
 


In [10]:
# Define 6-year period
subset_start = pd.Timestamp("2014-10-30")
subset_end   = pd.Timestamp("2020-9-29")

# Subset all datasets
data_6yr = {label: df.loc[subset_start:subset_end] for label, df in data.items()}

# Confirm shapes and date ranges
for label, df in data_6yr.items():
    print(f"{label}: {df.shape}, {df.index.min().date()} to {df.index.max().date()}")

Lake Fork River: (2162, 6), 2014-10-30 to 2020-09-29
Green River: (2162, 6), 2014-10-30 to 2020-09-29
Colorado River Potash: (2162, 6), 2014-10-30 to 2020-09-29
Colorado River Cisco: (2162, 6), 2014-10-30 to 2020-09-29
